In [3]:
import json

def inList(array , dictionary):
    for lval in array:
        for key, val in dictionary.items():
               for v in val:
                    if v in lval:
                          return key
                        
def mapDegInfo(array):
    if len(array) % 2 > 2:
        return 'Magistrale'
    else:
        return 'Triennale'
    
degreeInfo = {
    'Dottorato': ['Dottorato', 'Phd','PhD','PHD', 'P.h.D', 'P.h.d','P.H.D'],
    'Magistrale': ['Magistrale','magistrale','specialistica','Specialistica',"Master's","master's"],
    'Triennale': ['Triennale',"Bachelor's",'triennale']
}

tag = ['h3', 'span']
classes = ['education__item education__item--degree-info','pv-entity__comma-item','pv-entity__school-name t-16 t-black t-bold', 't-16 t-black t-bold','profile-section-card__title']

nameWrapperFile = 'nameWrapper.json'
nameWrapper = {}

jobLevelWrapperFile = 'jobLevelWrapper.json'
jobLevelWrapper = {}

with open(nameWrapperFile) as json_name_wrapper:
    nameWrapper = json.load(json_name_wrapper)

with open(jobLevelWrapperFile) as json_job_level_wrapper:
    jobLevelWrapper = json.load(json_job_level_wrapper)    

These are to accessories functions that i defined in order to scrape some extra contents out of the ones that i already accessed by processing html elements reading their text. In particular the first one that will be used as well in further chuncks of code, just loops over a list of elements and for each key value pairs of a dictionary whose values are lists, it checks wether an element of dictionary's values is contained and thus maps it with the corresponding key. In this case it is used in order to assign a degree type such as Master's, Bachelor's or P.h.d. MapDegInfo is used for all of those fields where the user did not enter the type of degree associated, basically it fetches the number of education fields inferring the type based on the number of fields fetched. To make it simple it is based on the assumption that if more than two fields are entered in education field, suppose a high school title and a bachelor's deegree, the functions asserts that if there is another field entered it must be a Master's deegree.       

In [6]:
import os
import numpy as np
import csv
import re
from bs4 import BeautifulSoup

files = os.listdir(".")

experiences = []
educations = []

profiles = []

for f in files:
    if f.endswith(".html"):
        with open(f, encoding='utf-8') as fp:
            soup = BeautifulSoup(fp)
            nameTags = (soup.find('h1', {'class':c}) for c in ['top-card-layout__title', 'text-heading-xlarge inline t-24 v-align-middle break-words'])
            nameText = next((nameTag for nameTag in nameTags if nameTag is not None), "")
            firstName = ""
            if nameText != "":
                firstName = nameText.get_text().strip().split(' ')[0].lower()
            gender = inList([firstName], nameWrapper)    
            regex = re.compile('.*experience.*')
            experienceSection = soup.find('section', {'class':regex})
            for el in experienceSection.find_all('span', {'class':'visually-hidden'}): el.decompose()
            experienceTags = (experienceSection.find_all(t, {'class': c}) for t in tag for c in classes)
            expText = next((exp for exp in experienceTags if len(exp) > 0), "")
            experience = [el.get_text().strip() for el in expText if el != ""]
            workLevel = inList(experience, jobLevelWrapper)
            regexEd = re.compile('.*education.*')
            educationSection = soup.find('section', {'class':regexEd})
            educationTags = (educationSection.find_all(tag[1], {'class':c}) for c in classes)
            edText = next((ed for ed in educationTags if len(ed) > 0), "")
            education = [el.get_text().strip() for el in edText if el != ""]
            educationTimes = [el.get_text() for el in educationSection.find_all('time')]
            infoDeg = inList(education, degreeInfo) if inList(education, degreeInfo) is not None else mapDegInfo(education)        
            profiles.append([experience, education, infoDeg, educationTimes, gender, workLevel])

print(profiles[:2])

[[['Training Specialist', 'Training Specialist', 'Legal Assistant Paralegal', 'Legal Affairs Specialist'], ['Master di I livello', 'Politiche di Sicurezza e Polizie Locali', 'Laurea Magistrale  LM in Giurisprudenza', 'Ragioniere perito commerciale - programmatore'], 'Magistrale', ['2015', '2009', '2014', '2004', '2009'], 'F', 'Quadro'], [["Esperienze di Lavoro all'Estero come Care Assistant,Barman,Waiter"], ['Scienze Politiche,Sociali ed Internazionali', 'Curriculum Sociologia', 'Diploma', 'Liceo Scientifico'], 'Triennale', ['2014', '2016', '2006', '2011'], 'M', 'Impiegato']]


In this first huge part of code above i built a scraper almost from scratch using Beautiful Soup which is a python library i used to pull out data from the html files of linkedin profiles i manually downloaded in the current local directory. In a nutshell what i did was looping over almost 500 profiles pages donwloaded from LinkedIn and for each file i extracted those element of interest in order to fatch data from education and work section. In order to do that i queried the significant html element by tag name and class name saving the text data i needed into a list of lists in order to write them into a csv file for persistency and then loading them into pandas df for the manipulation.  

In [12]:
#loggin into a .csv file

import csv

header = ['Job','Education','InfoDeg','DateRange', 'Gender', 'WorkLevel']

csv_file = 'linkedin_profiles2.csv'
with open(csv_file, 'w', newline='') as fp:
    writer = csv.writer(fp)
    writer.writerow(header)
    writer.writerows(profiles)

In [37]:
#reporting data
import pandas as pd
import numpy as np

df = pd.read_csv('linkedin_profiles2.csv', encoding='ISO-8859-1')
df.head()

,Job,Education,InfoDeg,DateRange,Gender,WorkLevel
0,"['Training Specialist', 'Training Specialist',...","['Master di I livello', 'Politiche di Sicurezz...",Magistrale,"['2015', '2009', '2014', '2004', '2009']",F,Quadro
1,"[""Esperienze di Lavoro all'Estero come Care As...","['Scienze Politiche,Sociali ed Internazionali'...",Triennale,"['2014', '2016', '2006', '2011']",M,Impiegato
2,"['docente di lettere', 'docente di lettere', '...",['Laurea Magistrale LM in Scienze Umanistiche...,Magistrale,"['2017', '2020', '2016', '2017', '2014', '2017']",F,Impiegato
3,"['Junior Software Developer', 'Promoter vendit...","['Laurea triennale', 'Ingegneria informatica',...",Triennale,"['2016', '2021']",M,Impiegato
4,"['IoT Edge Developer', 'Sviluppatore front-end']","['Laurea Magistrale LM', 'Ingegneria informat...",Magistrale,"['2019', '2020', '2015', '2018', '2010', '2015']",M,Impiegato


In [38]:
df.dtypes

Job          object
Education    object
InfoDeg      object
DateRange    object
Gender       object
WorkLevel    object
dtype: object

the dataset, whose first elements are showcased by the head() method of pandas library, does still contain data that are a bit raw. Indeed, the columns displayed incapsulate inside lists all of those data scraped from html pages of linkedin profile such as all jobs registered in the experience section, all education information that could go from high school to P.h.d degrees and each enrollment and graduation date related to each education title entered. From this first sight at data it appears to me that this dataset needs to be modified a bit in order to keep just the information of interest for this work such as the last (most significant) education titles among P.h.D, bachelor or master degrees from University of Bologna, the last job inserted in the list of jobs and the last date range involving enrollment and graduation date of the associated degree.

In [39]:
df.shape

(496, 6)

In [40]:
from ast import literal_eval

df.Job = df.Job.apply(literal_eval)
df.Education = df.Education.apply(literal_eval)
df.DateRange = df.DateRange.apply(literal_eval) 

in the cell above, since data fetched from the csv file containing linkedin profiles previously stored in arrays are perceived as one big string i applied a function in order to split those string back into the two lists of job, education and date range.  

In [41]:
import json

jobWrapperFile = 'jobWrapper.json'
educationWrapperFile = 'educationWrapper.json'

jobWrapper = {}
educationWrapper = {} 

with open(jobWrapperFile) as json_job_wrapper:
    jobWrapper = json.load(json_job_wrapper)

with open(educationWrapperFile) as json_edu_wrapper:
    educationWrapper = json.load(json_edu_wrapper)
    
print({k:jobWrapper[k] for k in list(jobWrapper.keys())[:2]})
print({k:educationWrapper[k] for k in list(educationWrapper.keys())[:2]})

{'Risorse umane': ['Responsabile Area Espansione', 'Senior Recruitment Officer', 'Analista Direzione Sviluppo Persone e Organizzazione', 'Training Specialist', 'HR'], 'Ristorazione': ['Assistente enologo', 'Barman', 'barista']}
{'Giurisprudenza': ['Giurisprudenza'], 'Scienze della comunicazione': ['Semiotica', 'Marketing', 'PubblicitÃ\xa0', 'Communication', 'Comunicazione', 'comunicazione', "comunicazione pubblica, d'impresa e pubblicitÃ\xa0", 'Mass media e Politica', 'Brand Strategy and Marketing', 'Scienze della comunicazione', 'Comunicazione e Digital Media', 'Scienze della Comunicazione', 'Comunicazione']}


As i anticipated previously, since education and job fields fetched from linkedin html pages could have entered with different naming even though they could be identified with a single category, i decided to compute some basic remapping. By the way in order to have a more clean and meaningful dataset to inspect i decided to map semantically similar jobs or degrees under one same branch. To clarify, for each profile whose work experience was Developer, Web Developer, Sviluppatore software and so on, i mapped those field under the same keyword Sviluppatore. In order to do so i wrapped into two json files key value pairs where i linked a list of synonims under a unique key. in the cell above it is wrapped the code through which i incapsulated the contents of those two files in two dictionaries whose two first elements i showcased to give a glimpse of the fields.        

In [42]:
def inList(array , dictionary):
    for lval in array:
        for key, val in dictionary.items():
               for v in val:
                    if v in lval:
                          return key
                        
def calcGraduationYears(degInfo, dateRanges):
    if dateRanges[0] < dateRanges[1]:
        return int(dateRanges[1])
    else:
        if degInfo == 'Triennale' or degInfo == 'Dottorato':
            return int(dateRanges[0]) + 3
        else:
            return int(dateRanges[0]) + 2

what comes then is the definition of another basic function other than inList that i already described which is the calcGraduationYears. the function basically watches if there are some missing graduationFields (given by the fact that the candidate is still studying to graduate) and infers them from the enrollment years by adding the expected time to graduate depending on the type of graduation.   

In [43]:
import warnings
import numpy as np
from functools import partial
warnings.filterwarnings('ignore')

df1 = df

mappedJobs = list(map(partial(inList, dictionary=jobWrapper), df1.Job))
mappedEdus = list(map(partial(inList, dictionary=educationWrapper), df1.Education))


df1['Job'] = mappedJobs
df1['Education'] = mappedEdus

df2 = df1[df1.DateRange.map(len) > 1]

enrollmentYears = [int(yearRange[0]) for yearRange in df2.DateRange]
graduationYears = list(map(calcGraduationYears, df2.InfoDeg , df2.DateRange))

df3 = df2

df3.drop('DateRange', axis = 1, inplace= True)
df3['enrollmentYears'] = enrollmentYears
df3['graduationYears'] = graduationYears

once the values are mapped i built another dataset switching the old values with the new ones whose first rows are showed in the following cells. After this small processing the dataset, fullfilled with more meaningful and easy-to-read data is ready to be scanned to get the first insights.  

In [44]:
df3.dropna(inplace=True)

print(df3.dtypes)
print(df3.shape)
df3.head()

Job                object
Education          object
InfoDeg            object
Gender             object
WorkLevel          object
enrollmentYears     int64
graduationYears     int64
dtype: object
(427, 7)


,Job,Education,InfoDeg,Gender,WorkLevel,enrollmentYears,graduationYears
0,Risorse umane,Giurisprudenza,Magistrale,F,Quadro,2015,2017
1,Ristorazione,Scienze politiche,Triennale,M,Impiegato,2014,2016
2,Insegnante,Lettere e Filosofia,Magistrale,F,Impiegato,2017,2020
3,Sviluppatore,Ingegneria Informatica,Triennale,M,Impiegato,2016,2021
4,Sviluppatore,Ingegneria Informatica,Magistrale,M,Impiegato,2019,2020


In [45]:
df3.to_csv('out.csv')

In [83]:
from pandas_profiling import ProfileReport

df3.reset_index(drop = True,inplace=True)

profile = ProfileReport(df3,missing_diagrams={'bar':False,'matrix':False,'heatmap':False,'dendrogram':False})
profile.to_file("report.html")

In the chunck of code above in order to display a rapresentation of the report of the data as fancy as possible i used the pandas-profiling module in order to save it in a html file. Basically what it does is to provide a more enriched rappresentation of the API describe() from pandas which is used as well to generate some descriptive statistics about the features of the dataset to give some insights about it, and it calculates how correlated the features are.  

In [2]:
import pandas as pd
import numpy as np
import seaborn as sbn


from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [6]:
df = pd.read_csv('out.csv')
df.head()

df1 = df.iloc[:, 1:]

print(df1.head())

transf_dtype = np.int32

InfoDeg = ['Triennale','Magistrale','Dottorato']
WorkLevel = ['Impiegato','Quadro','Dirigente']

ordinal = OrdinalEncoder(categories=[InfoDeg, WorkLevel])
categorical = OneHotEncoder(dtype = transf_dtype, sparse=False)

df_categorical = df1.drop(columns=['InfoDeg','WorkLevel','enrollmentYears','graduationYears']) 
df_ordinal = df1.loc[:,['InfoDeg','WorkLevel']]

cat = categorical.fit_transform(df_categorical) 
cols = categorical.get_feature_names()

adj_cols = list(map(lambda x : x.replace('x0_','').replace('x1_','').replace('x2_',''), cols))

X_cat = pd.DataFrame(cat, columns=adj_cols)

ordin = ordinal.fit_transform(df_ordinal) 
X_ordinal = pd.DataFrame(ordin, columns= ['InfoDeg','WorkLevel'])

X_p = pd.concat([X_cat, X_ordinal, df1.loc[:, ['enrollmentYears','graduationYears']]], axis=1)

print(X_cat.shape)
print(X_ordinal.shape)

X_p

             Job               Education     InfoDeg Gender  WorkLevel  \
0  Risorse umane          Giurisprudenza  Magistrale      F     Quadro   
1   Ristorazione       Scienze politiche   Triennale      M  Impiegato   
2     Insegnante     Lettere e Filosofia  Magistrale      F  Impiegato   
3   Sviluppatore  Ingegneria Informatica   Triennale      M  Impiegato   
4   Sviluppatore  Ingegneria Informatica  Magistrale      M  Impiegato   

   enrollmentYears  graduationYears  
0             2015             2017  
1             2014             2016  
2             2017             2020  
3             2016             2021  
4             2019             2020  
(427, 47)
(427, 2)


,Analista Finanziario/Economico,Analista informatico,Arte,Assistente sociale,Autore,Banca,Commerciale,Commercialista,Comunicazione,Consulente Finanziario/Economico,...,Matematica e Fisica,Scienze della comunicazione,Scienze politiche,Statistica,F,M,InfoDeg,WorkLevel,enrollmentYears,graduationYears
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1.0,1.0,2015,2017
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0.0,0.0,2014,2016
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1.0,0.0,2017,2020
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0.0,0.0,2016,2021
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1.0,0.0,2019,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0.0,0.0,2005,2009
423,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1.0,0.0,2019,2021
424,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0.0,0.0,1998,2010
425,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1.0,0.0,2019,2021


In [7]:
yw, Xw = X_p.loc[:, 'WorkLevel'], X_p.drop(columns = ['WorkLevel'])

print(yw.head())
print(Xw.head())

0    1.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: WorkLevel, dtype: float64
   Analista Finanziario/Economico  Analista informatico  Arte  \
0                               0                     0     0   
1                               0                     0     0   
2                               0                     0     0   
3                               0                     0     0   
4                               0                     0     0   

   Assistente sociale  Autore  Banca  Commerciale  Commercialista  \
0                   0       0      0            0               0   
1                   0       0      0            0               0   
2                   0       0      0            0               0   
3                   0       0      0            0               0   
4                   0       0      0            0               0   

   Comunicazione  Consulente Finanziario/Economico  ...  Lingue  \
0              0                                 0

In [9]:
Xw_train, Xw_test, yw_train, yw_test = train_test_split(Xw, yw)

Xw_train.head()

clf = DecisionTreeClassifier()
clf.fit()

,Analista Finanziario/Economico,Analista informatico,Arte,Assistente sociale,Autore,Banca,Commerciale,Commercialista,Comunicazione,Consulente Finanziario/Economico,...,Lingue,Matematica e Fisica,Scienze della comunicazione,Scienze politiche,Statistica,F,M,InfoDeg,enrollmentYears,graduationYears
220,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.0,2014,2016
148,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,1.0,2018,2021
79,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0.0,2017,2021
203,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0.0,2018,2021
126,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0.0,2012,2014
